#  **RAG with ChromaDB**

> **Project:** GuajiraClimateAgents  
> **Author:** Eder Arley León Gómez  
> **GitHub:** https://github.com/ealeongomez  
> **License:** MIT


In [ ]:
import sys, os
from pathlib import Path
from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [ ]:
PROJECT_ROOT = Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))

# Paths importantes
PDF_PATH = PROJECT_ROOT / "data" / "pdfs" / "4349.pdf"
EMBEDDINGS_PATH = PROJECT_ROOT / "data" / "embeddings"

load_dotenv(PROJECT_ROOT / ".env")

# Verificar que las API keys estén configuradas
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [ ]:
from src.utils.vector_store import VectorStore

In [ ]:
# Cargar el PDF
loader = PyPDFLoader(str(PDF_PATH))
documents = loader.load()


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

# Dividir los documentos
chunks = text_splitter.split_documents(documents)

In [ ]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"  # Modelo más económico y eficiente
)

vector_store = VectorStore(collection_name="Atlas_eolico_Colombia", embedding_function=embeddings)

In [ ]:
ids = vector_store.add_documents(chunks)

In [ ]:
# Ejemplo de búsqueda básica
query = "¿Cuál es el tema principal del documento?"

results = vector_store.similarity_search(query, k=3)

for i, doc in enumerate(results, 1):
    print(f"\n--- Resultado {i} ---")
    print(f"📄 Página: {doc.metadata.get('page', 'N/A')}")
    print(f"📝 Contenido:\n{doc.page_content[:400]}...")
    print()

In [ ]:
# Búsqueda con scores de similitud
query = "clima"

results_with_scores = vector_store.similarity_search_with_score(query, k=5)

for i, (doc, score) in enumerate(results_with_scores, 1):
    print(f"\n--- Resultado {i} (Score: {score:.4f}) ---")
    print(f"📄 Página: {doc.metadata.get('page', 'N/A')}")
    print(f"📝 Contenido:\n{doc.page_content[:300]}...")

## 6. Ejemplo de RAG completo

Combinamos la búsqueda vectorial con un LLM para responder preguntas.

In [ ]:
# Configurar el LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Crear el retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 4})

# Template para RAG
template = """Responde la pregunta basándote únicamente en el siguiente contexto:

{context}

Pregunta: {question}

Respuesta:"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Crear la cadena RAG
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ Cadena RAG configurada correctamente")

In [ ]:
# Hacer una pregunta usando RAG
question = "¿Cuál es el tema principal del documento?"

print(f"❓ Pregunta: {question}")
print("\n" + "=" * 60)
print("\n💬 Respuesta:")

response = rag_chain.invoke(question)
print(response)

In [ ]:
# Celda interactiva para hacer más preguntas
# Modifica la variable 'pregunta' y ejecuta esta celda

pregunta = "Escribe aquí tu pregunta sobre el documento"

print(f"❓ {pregunta}")
print("\n" + "-" * 60)
respuesta = rag_chain.invoke(pregunta)
print(f"\n💬 {respuesta}")